In [1]:
import numpy as np
from IPython.display import display, Image

In [2]:
input_tr = np.load('./Data/region1/training_inputs.npy')
output_tr = np.load('./Data/region1/training_set.npy')

In [3]:
display(input_tr.shape)
display(input_tr[0])
display(input_tr.dtype)
display(min(input_tr[0]), max(input_tr[0]), np.mean(input_tr[0]), np.std(input_tr[0]))
display(min(input_tr[1]), max(input_tr[1]), np.mean(input_tr[1]), np.std(input_tr[1]))

display(output_tr.shape)
display(output_tr[0])
display(output_tr.dtype)
display(min(output_tr[0]), max(output_tr[0]), np.mean(output_tr[0]), np.std(output_tr[0]))


In [4]:
from PIL import Image

In [5]:
def reshape_single_as_picture(input, size=31):
    return np.reshape(input, (size, size))

def as_single_picture(input, size=31):
    return Image.fromarray(reshape_single_as_picture(input, size), 'L')

In [6]:
display(as_single_picture(input_tr[0]))
display(as_single_picture(input_tr[2]))
display(as_single_picture(input_tr[700]))
display(as_single_picture(input_tr[-1]))
display(as_single_picture(input_tr[-2]))

display(input_tr[0]*1_000_000)

In [7]:
def downsample_input(dta):
    def process_single(pic):
        resized_pic = as_single_picture(pic).resize(size=(15, 15), resample=Image.BICUBIC)
        return np.array(resized_pic).reshape(15*15)
    return np.array([process_single(pic) for pic in dta])
    

In [8]:
downsample_input(input_tr)


In [9]:
def normalize_input(dta):
    return (dta/np.std(dta))

In [10]:
input_tr_processed = downsample_input(input_tr)
display(as_single_picture(input_tr_processed[0], 15))
display(as_single_picture(input_tr_processed[2], 15))
display(as_single_picture(input_tr_processed[-1], 15))
display(as_single_picture(input_tr_processed[-2], 15))

In [11]:
display(input_tr_processed[0], min(input_tr_processed[0]), max(input_tr_processed[0]))

In [12]:
_, output_shape = output_tr.shape
dta_len, input_shape = input_tr_processed.shape
display(dta_len, output_shape, input_shape)

In [13]:
import NDN3.NDNutils as NDNutils

In [14]:
hls = 40
d2x = 0.0005
l1 = 0.000001

hsm_params = NDNutils.ffnetwork_params(
    input_dims=[1, input_shape], 
    layer_sizes=[hls, 2*hls, output_shape],
    ei_layers=[0, hls // 2],
    normalization=[0], 
    layer_types=['normal','normal','normal'], 
    reg_list={
        'd2x':[d2x,None,None],
        'l1':[l1,None,None],
        'max':[None,None,100]})
hsm_params['weights_initializers']=['normal','normal','normal']
hsm_params['normalize_weights']=[1,0,0]

display(hsm_params)

In [39]:
import NDN3.NDN as NDN
hsm = NDN.NDN(hsm_params, noise_dist='poisson')

In [40]:
hsm.train(input_data=input_tr_processed, output_data=output_tr, train_indxs=np.array(range(dta_len)), test_indxs=np.array(range(dta_len)), learning_alg="adam", opt_params={'batch_size': 1, 'use_gpu': False})


In [41]:
results = hsm.generate_prediction(input_tr_processed)
display(results[0])
display(output_tr[0])

In [42]:
eval_info = hsm.eval_models(input_data=input_tr_processed, output_data=output_tr, data_indxs=np.array(range(dta_len)), nulladjusted=True)
eval_info

In [43]:
len(eval_info)

In [44]:
hsm = NDN.NDN(hsm_params, noise_dist='poisson')
hsm.train(input_data=input_tr_processed, output_data=output_tr, train_indxs=np.array(range(dta_len)), learning_alg="lbfgs")
eval_info = hsm.eval_models(input_data=input_tr_processed, output_data=output_tr, data_indxs=np.array(range(dta_len)), nulladjusted=True)
eval_info

In [45]:
results = hsm.generate_prediction(input_tr_processed)
display(results[0])
display(output_tr[0])